## Consolidating GVCFs

create sample list name

In [ ]:
%%R
list_VCF2.sample_map <- read_delim("/data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf/list_VCF1.list", col_names = FALSE, delim = "\t")

list_VCF2.sample_map %>% 
  mutate(filename1 = str_remove(X1 , "_recal3.vcf")) %>% 
  subset(select=c("filename1", "X1")) 

write.table(list_VCF2.sample_map, "/data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf/list_VCF2.sample_map", row.names = FALSE, col.names = FALSE, quote = FALSE, sep = "\t")


create exome interval file

In [ ]:
%%bash
curl -s "http://hgdownload.cse.ucsc.edu/goldenpath/hg19/database/refGene.txt.gz" |\
    gunzip -c | cut -f 3,5,6 | sort -t $'\t' -k1,1 -k2,2n | bedtools merge -i - > exome.bed
# There are some entries that are problamatic. Let's get rid of them.
grep $'chr[1-9,X,Y]\t\|chr[1-9][0-9]\t' exome.bed > exome2.bed

# this option is not working.
# Let's work on the whole genome and subset it to the exome list later.


Write the following in a file named consolidate_gvcf.sh.

In [ ]:
#!/bin/bash

# directory where input files are saved
dir1=$1

cd ${dir1}

refloc1=/data/humgen/burook/ref

/PHShome/bm363/bin/gatk-4.1.7.0/gatk GenomicsDBImport \
    --genomicsdb-workspace-path ${dir1}/SB_gdb \
    -L ${dir1}/exome2.bed \
    --sample-name-map ${dir1}/list_VCF2.sample_map

In [ ]:
# input data directory
dir1=/data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf

bsub -m "hna001" -n 24 -M 128000 -o "tmpc4.out" -e "tmpc4.err" "sh /PHShome/bm363/WES_analysis/notebooks/consolidate_gvcf.sh  ${dir1}"


## Joint Genotyping


Save the following joint genotying scrintp in a file named joint_variant_calling.sh

In [ ]:
#!/bin/bash

# directory where input files are saved
dir1=$1

cd ${dir1}

refloc1=/data/humgen/burook/ref
refloc2=/pub/genome_references/gatk-bundle/1.5/hg19

# Joint variant calling

#java -jar /PHShome/bm363/bin/gatk-4.1.7.0/gatk-package-4.1.7.0-local.jar \
#java -jar /PHShome/bm363/bin/gatk-4.1.7.0/gatk-package-4.1.7.0-spark.jar \
/PHShome/bm363/bin/gatk-4.1.7.0/gatk \
    GenotypeGVCFs \
    -R ${refloc1}/genome.fa \
    -V ${dir1}/SB_gdb \
    --dbsnp ${refloc1}/dbsnp_135.hg19.vcf \
    -O SysBio_exome_merged3.vcf

Now call this script from bash

In [ ]:
%%bash

# input data directory
dir1=/data/humgen/burook/sysbio_exome/gatk_indiv_vcf
# output data directory
dir2=/data/humgen/burook/sysbio_exome/gatk_merged_vcf

bsub -m "hna001" -n 24 -M 64000 -o "tmp52.out" -e "tmp52.err" "sh joint_variant_calling.sh  ${dir1}"

# now let's move the output to the desired directory
mv ${dir1}/SysBio_exome_merged1.vcf ${dir2}/

## Variant Recalibration

## Subset final VCF to the target exome list